In [67]:
import pyspark
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as F

In [68]:
spark = SparkSession.builder.getOrCreate()

In [69]:
case = spark.read.csv(r'/Users/garrettwilliford/Downloads/case.csv', sep=",", header=True, inferSchema=True)
dept = spark.read.csv(r'/Users/garrettwilliford/Downloads/dept.csv', sep=",", header=True, inferSchema=True)
source = spark.read.csv(r'/Users/garrettwilliford/Downloads/source.csv', sep=",", header=True, inferSchema=True)

In [71]:
case.write.json('/Users/garrettwilliford/Desktop/codeup-data-science/ds-methodologies-exercises/spark/case_json')
dept.write.json('/Users/garrettwilliford/Desktop/codeup-data-science/ds-methodologies-exercises/spark/dept_json')
source.write.json('/Users/garrettwilliford/Desktop/codeup-data-science/ds-methodologies-exercises/spark/source_json')

In [72]:
case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [18]:
#How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?
#How many Stray Animal cases are there?
#How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?
#Convert the council_district column to a string column.
#Extract the year from the case_closed_date column.
#Convert num_days_late from days to hours in new columns num_hours_late.
#Join the case data with the source and department data.
#Are there any cases that do not have a request source?
#What are the top 10 service request types in terms of number of requests?
#What are the top 10 service request types in terms of average days late?
#Does number of days late depend on department?
#How do number of days late depend on department and request type?

In [73]:
case_not_closed = case.where(case.case_status == 'Open')

In [74]:
is_animal = case.where(case.service_request_type == 'Stray Animal')

In [75]:
#Number of stray animal cases
is_animal.count()

26760

In [76]:
field_operations = case.where((case.dept_division == 'Field Operations') & (case.service_request_type != 'Officer Standby'))

In [77]:
#Number of field_operations cases not classified as Officer Standby
field_operations.count()

113902

In [118]:
case.select((case.num_days_late * 24)).show(5)


+--------------------+
|(num_days_late * 24)|
+--------------------+
|      -23964.2102784|
| -48.302500007999996|
|       -72.536111112|
|       -360.27555552|
|   8.931944448000001|
+--------------------+
only showing top 5 rows



In [119]:
#vertical is pretty cool, i very much enjoy how the output does not look like garbage
data = case.join(source, on = 'source_id').join(dept, on = 'dept_division')
data.select('*').show(vertical = True)

-RECORD 0--------------------------------------
 dept_division          | Field Operations     
 source_id              | svcCRMLS             
 case_id                | 1014127332           
 case_opened_date       | 1/1/18 0:42          
 case_closed_date       | 1/1/18 12:29         
 SLA_due_date           | 9/26/20 0:42         
 case_late              | NO                   
 num_days_late          | -998.5087616000001   
 case_closed            | YES                  
 service_request_type   | Stray Animal         
 SLA_days               | 999.0                
 case_status            | Closed               
 request_address        | 2315  EL PASO ST,... 
 council_district       | 5                    
 source_username        | svcCRMLS             
 dept_name              | Animal Care Services 
 standardized_dept_name | Animal Care Services 
 dept_subject_to_SLA    | YES                  
-RECORD 1--------------------------------------
 dept_division          | Storm Water   

Column<b'9/9/17 9:56'>

In [122]:
case.select((case.num_days_late * 24)).show()

+--------------------+
|(num_days_late * 24)|
+--------------------+
|      -23964.2102784|
| -48.302500007999996|
|       -72.536111112|
|       -360.27555552|
|   8.931944448000001|
|  -713.8555555199999|
|       -352.96166664|
|       -352.95888888|
|       -352.95888888|
|       -352.95583344|
|       -352.95583344|
|       -352.95277776|
| -352.95000000000005|
|       -352.95277776|
| -352.94972232000003|
| -352.94138879999997|
|       -352.94416656|
|       -352.93833336|
|       -352.93833336|
|        -352.9355556|
+--------------------+
only showing top 20 rows



In [123]:
dept_data = case.join(source, on = 'source_id', how = 'inner').join(dept, on = 'dept_division')
dept_data.select('*').show(1, vertical = True)

-RECORD 0--------------------------------------
 dept_division          | Field Operations     
 source_id              | svcCRMLS             
 case_id                | 1014127332           
 case_opened_date       | 1/1/18 0:42          
 case_closed_date       | 1/1/18 12:29         
 SLA_due_date           | 9/26/20 0:42         
 case_late              | NO                   
 num_days_late          | -998.5087616000001   
 case_closed            | YES                  
 service_request_type   | Stray Animal         
 SLA_days               | 999.0                
 case_status            | Closed               
 request_address        | 2315  EL PASO ST,... 
 council_district       | 5                    
 source_username        | svcCRMLS             
 dept_name              | Animal Care Services 
 standardized_dept_name | Animal Care Services 
 dept_subject_to_SLA    | YES                  
only showing top 1 row



In [138]:
#df.where(df.source_id == '').show()
time_over = F.when(F.isnull('case_closed_date'), F.datediff(max_date, case.SLA_due_date))\
                .otherwise(F.datediff(case.case_closed_date  , case.SLA_due_date)).alias('overdue_time')
case.select('*', time_over).where(time_over == F.lit(case.select(F.max(time_over)).head()[0])).show(1, vertical = True)


(0 rows)



In [132]:
case.groupBy('service_request_type').count().sort(F.desc('count')).show(10)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           No Pickup|86855|
|Overgrown Yard/Trash|65895|
|        Bandit Signs|32910|
|        Damaged Cart|30338|
|Front Or Side Yar...|28794|
|        Stray Animal|26760|
|Aggressive Animal...|24882|
|Cart Exchange Req...|22024|
|Junk Vehicle On P...|21473|
|     Pot Hole Repair|20616|
+--------------------+-----+
only showing top 10 rows



In [133]:
case.groupBy('service_request_type').agg(F.mean(time_over).alias('mean_days_late')).sort(F.desc('mean_days_late')).show(10)

+--------------------+--------------+
|service_request_type|mean_days_late|
+--------------------+--------------+
|Minimum Housing-O...|          null|
|        Tree Removal|          null|
| Service Information|          null|
|    Sign Maintenance|          null|
|Park Building Mai...|          null|
|Brush Property Da...|          null|
|Graffiti: Private...|          null|
|Guardrail- New Re...|          null|
|Markings Installa...|          null|
|Traffic Sign Graf...|          null|
+--------------------+--------------+
only showing top 10 rows



In [134]:
case.groupBy('dept_division').agg(F.mean(time_over).alias('avg_days_late')).sort(F.desc('avg_days_late')).show()

+--------------------+-------------+
|       dept_division|avg_days_late|
+--------------------+-------------+
|    Field Operations|         null|
|       Miscellaneous|         null|
|         Solid Waste|         null|
|          District 7|         null|
|             Streets|         null|
|    Waste Collection|         null|
|Code Enforcement ...|         null|
|         District 10|         null|
|   Dangerous Premise|         null|
|     311 Call Center|         null|
|              Vector|         null|
|Traffic Engineeri...|         null|
|Code Enforcement ...|         null|
|        Reservations|         null|
|Engineering Division|         null|
|             Signals|         null|
|         Storm Water|         null|
|Director's Office...|         null|
|          District 2|         null|
|               Shops|         null|
+--------------------+-------------+
only showing top 20 rows



In [136]:
case.groupBy('dept_division', 'service_request_type').agg(F.mean(time_over).alias('avg_days_late')).sort(F.desc('avg_days_late')).show()

+--------------------+--------------------+-------------+
|       dept_division|service_request_type|avg_days_late|
+--------------------+--------------------+-------------+
|    Code Enforcement|Overgrown Yard/Trash|         null|
|    Shops (Internal)|Major Park Improv...|         null|
|             Signals|Signal Timing Mod...|         null|
| Food Establishments|      Food Poisoning|         null|
|Storm Water Engin...|Illegal Floodplai...|         null|
|    Code Enforcement|Temporary Obstruc...|         null|
|    Field Operations|Aggressive Animal...|         null|
|     311 Call Center|           Complaint|         null|
|Traffic Engineeri...|Flashing Beacon N...|         null|
|    Shops (Internal)|         Playgrounds|         null|
|Traffic Engineeri...|       Parking Issue|         null|
|    Code Enforcement|     Permits, Fences|         null|
|    Waste Collection|Organics Info / L...|         null|
|             Streets| Rebarb Sticking Out|         null|
|    Code Enfo